In [ ]:
import shapely.geometry
import pandas as pd
import numpy as np
import importlib
import matplotlib.pyplot as plt
from datetime import datetime
import time
import copy

import math
import IPython
import pickle
import os

In [ ]:
# ais
with open('./files/ais_0204.pkl', 'rb') as f:
    ais = pd.read_pickle(f)

with open('./files/ais_ts_list.pkl', 'rb') as f:
    ais_ts_list = pd.read_pickle(f)

with open('./files/ais_dict.pkl', 'rb') as f:
    ais_dict = pd.read_pickle(f)

In [ ]:
#좌표 v가 속하는 그리드의 키값 반환
def find_grid_key(llist, v):
    result = -1
    for i in range(0, len(llist)-1):
        if llist[i] <= v and llist[i+1] >= v:
            return llist[i]
    return result

# 그리드 별 웨이트 정보 할당

In [ ]:
#그리드맵에 NESW방향 웨이트 정보 할당

def insert_data_traffic(grid_map, grid_lat, grid_lon, grid_lat_lon_keys):
    progress = []
    mmsis = list(set(ais_ts_list[0]["MMSI"].tolist()))

    for i, mmsi in enumerate(mmsis):

        pais = ais_dict[mmsi]
        plat = copy.deepcopy(pais.위도.tolist())
        plon = copy.deepcopy(pais.경도.tolist())

        klat = plat.pop(0)
        klon = plon.pop(0)

        key = '(%f,%f)'%(find_grid_key(grid_lat, klat), find_grid_key(grid_lon, klon))
        if key not in grid_lat_lon_keys:
            continue

        while(len(plat) > 0):
            tklat = plat.pop(0)
            tklon = plon.pop(0)
            tkey = '(%f,%f)'%(find_grid_key(grid_lat, tklat), find_grid_key(grid_lon, tklon))
            if tkey not in grid_lat_lon_keys:
                continue

            if(key == tkey): #그리드 간 이동이 있을 때만 방향확인
                continue
            if (klat < tklat):
                grid_map[key][3] += 1
            if (klon < tklon):
                grid_map[key][4] += 1
            if (klat > tklat):
                grid_map[key][5] += 1
            if (klon > tklon):
                grid_map[key][6] += 1


            klat = tklat
            klon = tklon
            key = '(%f,%f)'%(find_grid_key(grid_lat, klat), find_grid_key(grid_lon, klon))

In [ ]:
def direction_weight(grid_map, grid_lat, grid_lon, grid_lat_lon_keys, destination_lat, destination_lon, score_dic):

    direction_value = {}

    for i in range(len(grid_lat)):
        for j in range(len(grid_lon)):
            key = '(%f,%f)'%(grid_lat[i], grid_lon[j])
            if key not in grid_lat_lon_keys:
                continue

            if (grid_lat[i] > destination_lat) and (grid_lon[j] > destination_lon): # 남서쪽
                score_dic[key][0] = grid_map[key][5] + grid_map[key][6]

            elif (grid_lat[i] > destination_lat) and (grid_lon[j] < destination_lon): # 남동쪽
                score_dic[key][0] = grid_map[key][4] + grid_map[key][5]

            elif (grid_lat[i] < destination_lat) and (grid_lon[j] < destination_lon): # 북동쪽
                score_dic[key][0] = grid_map[key][3] + grid_map[key][4]

            elif (grid_lat[i] < destination_lat) and (grid_lon[j] > destination_lon): # 북서쪽
                score_dic[key][0] = grid_map[key][3] + grid_map[key][6]

            elif (grid_lat[i] > destination_lat) and (grid_lon[j] == destination_lon): # 남쪽
                score_dic[key][0] = 2*grid_map[key][5]

            elif (grid_lat[i] < destination_lat) and (grid_lon[j] == destination_lon): # 북쪽
                score_dic[key][0] = 2*grid_map[key][3]

            elif (grid_lat[i] == destination_lat) and (grid_lon[j] < destination_lon): # 동쪽
                score_dic[key][0] = 2*grid_map[key][4]

            elif (grid_lat[i] == destination_lat) and (grid_lon[j] > destination_lon): # 서쪽
                score_dic[key][0] = 2*grid_map[key][6]

# 트래픽 값 퀀타일 적용, 스코어링

In [ ]:
def quantile_traffic_volume(dic):
    value_list = []

    for key in dic.keys():
        lat, lon = eval(key)
        value_list.append(dic[key][0])

    quantile_values = np.quantile(value_list, [0.3, 0.6])
    return quantile_values

In [ ]:
def score_traffic_volume(dic):
    Q = quantile_traffic_volume(dic)
    for key in dic.keys():
        lat, lon = eval(key)

        value = dic[key][0]
        if value == 0:
                score_value = 4
        elif 0 < value <= Q[0]:
                score_value = 3
        elif Q[0] < value <= Q[1]:
                score_value = 2
        else:
                score_value = 1
        dic[key][3] = score_value